# Deploy COVID-19 Prediction Model into Azure ML Workspce

In [1]:
from platform import python_version
print(python_version())

3.7.3


In [2]:
from azureml.core import Workspace, Experiment, Run
from azureml.core.model import Model

In [3]:
import azureml.core
from azureml.core import Workspace

In [4]:
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.4.0


### 1. Get Workspace Information

Make sure config.json file have below listed Azure ML workspce value 

{
   "subscription_id": "AzureML_Subscription_ID",
   
  "resource_group": "ResourceGroupName",
  
   "workspace_name": "AzureML workspace Name"
}


In [8]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication(tenant_id="<Your Azure tenant Id>")

ws = Workspace(subscription_id="<Your Azure Sub scription Id>",
               resource_group="<Resource Group>",
               workspace_name="<Your Workspace Name>",
               auth=interactive_auth)

WARNING - Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"


Performing interactive authentication. Please follow the instructions on the terminal.


WARNING - You have logged in. Now let us find all the subscriptions to which you have access...


Interactive authentication successfully completed.


### 2. Register the model

In [11]:
# Tip: When model_path is set to a directory, you can use the child_paths parameter to include
#      only some of the files from the directory
model = Model.register(model_path = "Covid_Final_Best_model.h5",
                       model_name = "Covid19Prediction",
                       description = "COVID19 Prediction using xception model",
                       workspace = ws)

Registering model Covid19Prediction


### 3. Define inference configuration

In [12]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies

# Create the environment
myenv = Environment(name="myenv")
conda_dep = CondaDependencies()

# Define the packages needed by the model and scripts
conda_dep.add_conda_package("pandas")
conda_dep.add_conda_package("numpy")
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("tensorflow")
conda_dep.add_conda_package("keras")
conda_dep.add_conda_package("opencv")
# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")


# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

inference_config = InferenceConfig(entry_script="modelscore.py",
                                   environment=myenv)

### 4. Deploy the Model

In [13]:
#from azureml.core.webservice import AciWebservice, AksWebservice, LocalWebservice
from azureml.core.webservice import AciWebservice,Webservice

In [14]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 4)

In [15]:
model = Model(ws, name='Covid19Prediction')

In [16]:

service = Model.deploy(ws, 'covid19webservice', [model], inference_config, deployment_config)

service.wait_for_deployment(True)
print(service.state)
print("scoring URI: " + service.scoring_uri)

Running.................................................................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
Healthy
scoring URI: http://25d27dd2-e7ee-4388-b7ff-509c009fe991.westus.azurecontainer.io/score


#### Update the model

In [ ]:
service_name = 'covid19webservice'
# Retrieve existing service.
service = Webservice(name=service_name, workspace=ws)

In [ ]:
# Update to new model(s).
service.update(models=[model], inference_config=inference_config)
print(service.state)
print(service.get_logs())

In [ ]:
service.auth_enabled

In [ ]:
print(service.swagger_uri)

In [ ]:
primary, secondary = service.get_keys()
print(primary)

In [ ]:
deployment_config

In [ ]:


# Get the deployed service
service = AciWebservice(name='aciwebservice', workspace=ws)

# Check its state
print(service.state)

In [ ]:
service.get_logs()

### Request-response consumption

In [17]:
import requests
import json
import base64


In [18]:
filePath=r"A:\My Learning\Python\CTS-AI\COVID-19\Dataset\Covid19.jpg"

In [19]:
with open(filePath, "rb") as image:
    f = image.read()
    b = bytearray(f)

In [20]:
str = base64.b64encode(b)

In [21]:
str=str.decode("utf-8")

In [22]:
json_data={'data':str}
json_data= json.dumps(json_data)

In [23]:
jsonData=json.loads(json_data)

In [24]:

test_samples = bytes(json_data, encoding='utf8')

# predict using the deployed model
result = service.run(input_data=test_samples)
print(result)


{'label': 'POSITIVE', 'prediction': '0.9937319', 'elapsed_time': 4.826961517333984}


### 5. Test Deployed Model ACI Service

In [25]:
import requests
import json

scoring_uri = 'http://25d27dd2-e7ee-4388-b7ff-509c009fe991.westus.azurecontainer.io/score'
headers = {'Content-Type':'application/json'}

# predict using the deployed model
result = service.run(input_data=test_samples)

response = requests.post(scoring_uri, data=json_data, headers=headers)
print(response.status_code)
print(response.elapsed)
print(response.json())

200
0:00:02.075431
{'label': 'POSITIVE', 'prediction': '0.9937319', 'elapsed_time': 0.3000154495239258}
